In [2]:
import numpy as np
import pandas as pd
import pickle

In [2]:
path = 'D:/Emily/Competition/T-brain/CreditCard_FraudDetection/'

## 匯入原始訓練、測試資料

In [3]:
df_train = pd.read_csv(path + 'dataset/original/train.csv')
df_test = pd.read_csv(path + 'dataset/original/test.csv')

In [4]:
txkey = df_test['txkey']

## Building Model

In [5]:
# training data set
df_train_trans = pd.read_csv(path + 'dataset/transformed/df_train_trans.csv')

X = df_train_trans.iloc[:, df_train_trans.columns != 'fraud_ind'].drop(columns = ['Unnamed: 0'])
y = df_train_trans['fraud_ind']

# test data set
df_test_trans = pd.read_csv(path + 'dataset/transformed/df_test_trans.csv', index_col = False )
df_test_trans = df_test_trans[['WOE_ecfg', 'WOE_flbmk', 'WOE_flg_3dsmk', 'WOE_insfg', 'WOE_ovrlt', 'WOE_loctm_H', 'WOE_acqic', 'WOE_bacno', 'WOE_cano', 'WOE_contp', 'WOE_csmcu', 'WOE_etymd', 'WOE_hcefg', 'WOE_mcc', 'WOE_mchno', 'WOE_scity', 'WOE_stocn', 'WOE_stscd', 'conam', 'iterm']]


## BaggingClassifier

In [81]:
from sklearn.datasets import make_classification
# X, y = make_classification(n_samples=10000, n_features=2, n_informative=2,
#                            n_redundant=0, n_repeated=0, n_classes=3,
#                            n_clusters_per_class=1,
#                            weights=[0.01, 0.05, 0.94], class_sep=0.8,
#                            random_state=0)
X, y = make_classification(n_samples=len(df_train_trans), n_features=len(X.columns), n_informative=2,
                           n_redundant=0, n_repeated=0, n_classes=len(y.fraud_ind.unique()),
                           n_clusters_per_class=1,
                           weights=[0.98, 0.02], class_sep=0.8,
                           random_state=0)



from sklearn.model_selection import train_test_split
from sklearn.metrics import balanced_accuracy_score
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)
bc = BaggingClassifier(base_estimator=DecisionTreeClassifier(),
                       random_state=0)
bc.fit(X_train, y_train)

y_pred = bc.predict(X_test)
score_BaggingClassifier = balanced_accuracy_score(y_test, y_pred)
print('score_BaggingClassifier: ' + str(score_BaggingClassifier))


# -- Model saving
pickle.dump(bc, open('./models/classifier/BaggingClassifier_DecisionTree.pkl', 'wb'))



score_BaggingClassifier: 0.9977973568281938


In [82]:
pred_bc = bc.predict(df_test_trans)

result_bc = pd.DataFrame(txkey,columns = ['txkey'])
result_bc['fraud_ind'] = pred_bc.tolist()
result_bc.to_csv(path + 'result/result_pred_bc.csv', index = False)

## BalancedBaggingClassifier

In [83]:
from imblearn.ensemble import BalancedBaggingClassifier
# https://imbalanced-learn.readthedocs.io/en/stable/index.html
# Scikit learn 0.21 needed

bbc = BalancedBaggingClassifier(base_estimator=DecisionTreeClassifier(),
                                sampling_strategy='auto',
                                replacement=False,
                                random_state=0)
bbc.fit(X_train, y_train)

y_pred = bbc.predict(X_test)
score_BalancedBaggingClassifier = balanced_accuracy_score(y_test, y_pred)
print('score_BalancedBaggingClassifier: ' + str(score_BalancedBaggingClassifier))

# -- Model saving
pickle.dump(bbc, open('./models/classifier/BalancedBaggingClassifier.pkl', 'wb'))



score_BalancedBaggingClassifier: 0.9955947136563876


In [84]:
pred_bbc = bbc.predict(df_test_trans)

result_bbc = pd.DataFrame(txkey,columns = ['txkey'])
result_bbc['fraud_ind'] = pred_bbc.tolist()
result_bbc.to_csv(path + 'result/result_pred_bbc.csv', index = False)

## BalancedRandomForestClassifier

In [85]:
from imblearn.ensemble import BalancedRandomForestClassifier
brf = BalancedRandomForestClassifier(n_estimators=100, random_state=0)
brf.fit(X_train, y_train)

y_pred = brf.predict(X_test)
score_BalancedRandomForestClassifier =balanced_accuracy_score(y_test, y_pred)
print('score_BalancedRandomForestClassifier: ' + str(score_BalancedRandomForestClassifier))

fea_imp_BalancedRandomForestClassifier = brf.feature_importances_
print('fea_imp_BalancedRandomForestClassifier: ' + str(fea_imp_BalancedRandomForestClassifier))

# -- Model saving
pickle.dump(brf, open('./models/classifier/BalancedRandomForestClassifier.pkl', 'wb'))

score_BalancedRandomForestClassifier: 0.9955947136563876
fea_imp_BalancedRandomForestClassifier: [0.01709301 0.01188061 0.0116047  0.01246991 0.00593986 0.05144099
 0.39002678 0.01178568 0.00403756 0.00846259 0.38607645 0.00742698
 0.01066666 0.00678281 0.01126271 0.01037393 0.01254423 0.00724899
 0.00684072 0.01603483]


In [86]:
pred_brf = brf.predict(df_test_trans)

result_brf = pd.DataFrame(txkey,columns = ['txkey'])
result_brf['fraud_ind'] = pred_brf.tolist()
result_brf.to_csv(path + 'result/result_pred_brf.csv', index = False)

In [38]:
balanced_accuracy_score(y_test, y_pred)

0.8351677558630584

In [39]:
brf.feature_importances_

array([0.02472163, 0.02465619, 0.02394815, 0.02450324, 0.02452545,
       0.02450178, 0.02405066, 0.02476033, 0.02438878, 0.02435631,
       0.02459406, 0.02413857, 0.17948591, 0.0249485 , 0.02450111,
       0.02434284, 0.02451048, 0.02421102, 0.38062339, 0.0242316 ])

## 資料做imbalence處理

In [1]:
from models import *

from sklearn.metrics import *
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.utils import class_weight



Using TensorFlow backend.


In [7]:
# Class weight
class_weights = class_weight.compute_class_weight('balanced', np.unique(y), y)

# Training/validation set split
tr_X, vl_X, tr_y, vl_y = train_test_split(X, y, test_size=0.3, stratify = y, random_state=42)

# del X, y

In [12]:
print('len(y)= '+ str(len(y)))
print('sum(y)= '+ str(sum(y)))
print('sum(tr_y)+sum(vl_y)= ' + str(sum(tr_y)+sum(vl_y)))
print('class_weights= ' +str(class_weights))

print('(len(y)-sum(y))*class_weights[0]= ' +str((len(y)-sum(y))*class_weights[0]))
print('sum(y)*class_weights[1]= ' +str(sum(y)*class_weights[1]))
print('total_balanced= ' + str(((len(y)-sum(y))*class_weights[0]) + (sum(y)*class_weights[1])))

len(y)= 1521787
sum(y)= 20355
sum(tr_y)+sum(vl_y)= 20355
class_weights= [ 0.50677853 37.38115942]
(len(y)-sum(y))*class_weights[0]= 760893.5
sum(y)*class_weights[1]= 760893.5
total_balanced= 1521787.0


## XGB: balanced

In [8]:
# -- scorer as loss
def f2_score(preds, y):
    # perform kappa calculation
    return 'f2_score', fbeta_score(y.astype(int), preds.astype(int), beta=2,
                                   average='binary', sample_weight=[class_weights[i] for i in tr_y.values])

In [100]:
model = xgb_model_config_initialise()
model = ensemble_model_initialise(base_estimator=model)
model.fit(tr_X, tr_y)



# -- Model saving
# pickle.dump(model, open('./models/classifier/ensemble_xgb_f2_scorer.pkl', 'wb'))
pickle.dump(model, open('./models/classifier/ensemble_xgb_1.pkl', 'wb'))

model

EasyEnsembleClassifier(base_estimator=XGBClassifier(base_score=0.5,
                                                    booster='gbtree',
                                                    colsample_bylevel=1,
                                                    colsample_bynode=1,
                                                    colsample_bytree=0.5,
                                                    gamma=0, learning_rate=0.1,
                                                    max_delta_step=1,
                                                    max_depth=8,
                                                    min_child_weight=1,
                                                    missing=None,
                                                    n_estimators=500, n_jobs=1,
                                                    nthread=None,
                                                    objective='binary:logistic',
                                                    predictor='

In [101]:
# Validation

vL_y_prob = model.predict_proba(vl_X)
vL_y_pred = model.predict(vl_X)

cm = confusion_matrix(vl_y.values, vL_y_pred)
precision, recall, f1score, _ = precision_recall_fscore_support(vl_y.values, vL_y_pred, average='binary',
                                                                sample_weight=[class_weights[i] for i in vl_y.values])  # weighted
roc_area = roc_auc_score(vl_y.values, vL_y_prob[:, 1])
print('Confusion matrix: ', cm)
print('Precision: {p:.3f}'.format(p=precision))
print('Recall: {r:.3f}'.format(r=recall))
print('F1score: {f:.3f}'.format(f=f1score))
print('ROC curve under area: {roc:.3f}'.format(roc=roc_area))

Confusion matrix:  [[437313  13117]
 [   131   5976]]
Precision: 0.971
Recall: 0.979
F1score: 0.975
ROC curve under area: 0.996


In [60]:
# Validation: cut probability
vL_y_prob = model.predict_proba(vl_X)
vL_y_prob_list = []
for i in range(0, len(vL_y_prob)):
    vL_y_prob_list.append(vL_y_prob[i][1])

df_prob_vL_y = pd.DataFrame(vL_y_prob_list,columns = ['prob'])
# df_prob_vL_y.values


vL_y_pred = np.where((df_prob_vL_y.values > 0.956) == False, 0, 1 )
print(100*sum(vL_y_pred)/len(vL_y_pred))
print(sum(vL_y_pred))
print(len(vL_y_pred))
print(421665*(1.33/100)) # percentage of target=1 from training data: 5608.1445

vL_y_pred_list= []
for i in range(0,len(vL_y_pred)):
    vL_y_pred_list.append(vL_y_pred[i][0])

cm = confusion_matrix(vl_y.values, vL_y_pred_list)
precision, recall, f1score, _ = precision_recall_fscore_support(vl_y.values, vL_y_pred_list, average='binary',
                                                                sample_weight=[class_weights[i] for i in vl_y.values])  # weighted
roc_area = roc_auc_score(vl_y.values, vL_y_prob[:, 1])
print('Confusion matrix: ', cm)
print('Precision: {p:.3f}'.format(p=precision))
print('Recall: {r:.3f}'.format(r=recall))
print('F1score: {f:.3f}'.format(f=f1score))
print('ROC curve under area: {roc:.3f}'.format(roc=roc_area))

Confusion matrix:  [[447439   2991]
 [   688   5419]]
Precision: 0.993
Recall: 0.887
F1score: 0.937
ROC curve under area: 0.996


In [51]:
# pred_ensem_xgb_f2 =model.predict(df_test_trans) 

# result = pd.DataFrame(txkey,columns = ['txkey'])
# result['fraud_ind'] = pred_ensem_xgb_f2.tolist()

# result.to_csv(path + 'result/result_pred_ensem_xgb_f2.csv', index = False)

In [20]:
# -- load
# model = pickle.load(open('./models/classifier/ensemble_xgb_1.pkl', "rb"))
prob_xgb = model.predict_proba(df_test_trans) 



prob_xgb_list = []
for i in range(0, len(prob_xgb)):
    prob_xgb_list.append(prob_xgb[i][1])

df_prob_xgb = pd.DataFrame(prob_xgb_list,columns = ['prob'])
df_prob_xgb.values

array([[0.08004415],
       [0.14666178],
       [0.09698226],
       ...,
       [0.00023011],
       [0.00129887],
       [0.00231623]])

In [31]:
pred_xgb = np.where((df_prob_xgb.values > 0.956) == False, 0, 1 )
print(100*sum(pred_xgb)/len(pred_xgb))
print(sum(pred_xgb))
print(len(pred_xgb))
print(421665*(1.33/100)) # percentage of target=1 from training data: 5608.1445

[1.33921478]
[5647]
421665
5608.1445


In [36]:
fraud_ind_list= []
for i in range(0,len(pred_xgb)):
    fraud_ind_list.append(pred_xgb[i][0])

result_model = pd.DataFrame(txkey,columns = ['txkey'])
result_model['fraud_ind'] = fraud_ind_list

result_model.to_csv(path + 'result/result_pred_XGB_cutprob.csv', index = False)

## XGB: imbalanced

In [21]:
model_1 = xgb.XGBClassifier(**config)
model_1.fit(tr_X, tr_y)

# -- Model saving
pickle.dump(model_1, open('./models/classifier/xgb.pkl', 'wb'))

model_1

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.5,
              eval_set=(         WOE_ecfg  WOE_flbmk  WOE_flg_3dsmk  WOE_insfg  WOE_ovrlt  \
1195536 -1.679063  -0.002313       0.037784   0.026741  -0.018542   
427655   1.249735  -0.002313       0.037784   0.026741  -0.018542   
579151  -1.679063  -0.002313       0.037784   0.026741  -0.018542   
221857  -1.679063  -0.002313       0.037784   0.0267...
Name: fraud_ind, Length: 152179, dtype: int64),
              feval=<function f2_score at 0x000001BE16D69F28>, gamma=0,
              learning_rate=0.1, max_delta_step=1, max_depth=8,
              min_child_weight=1, missing=None, n_estimators=50, n_jobs=-1,
              nthread=None, objective='binary:logistic',
              predictor='cpu_predictor', random_state=42, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=1, seed=None, silent=True,
              subsample=0.8, verbose=0, verbosity=1)

In [32]:
# -- load
# model_1 = pickle.load(open('./models/classifier/xgb.pkl', "rb"))

# Validation
vL_y_pred = model_1.predict(vl_X)
vL_y_prob = model_1.predict_proba(vl_X)

cm = confusion_matrix(vl_y.values, vL_y_pred)
precision, recall, f1score, _ = precision_recall_fscore_support(vl_y.values, vL_y_pred, average='binary',
                                                                sample_weight=[class_weights[i] for i in vl_y.values])  # weighted
roc_area = roc_auc_score(vl_y.values, vL_y_prob[:, 1])
print('Confusion matrix: ', cm)
print('Precision: {p:.3f}'.format(p=precision))
print('Recall: {r:.3f}'.format(r=recall))
print('F1score: {f:.3f}'.format(f=f1score))
print('ROC curve under area: {roc:.3f}'.format(roc=roc_area))

Confusion matrix:  [[149980    199]
 [   567   1433]]
Precision: 0.998
Recall: 0.717
F1score: 0.834
ROC curve under area: 0.996


In [33]:
pred_xgb = model_1.predict(df_test_trans) 

result = pd.DataFrame(txkey,columns = ['txkey'])
result['fraud_ind'] = pred_xgb.tolist()

result.to_csv(path + 'result/result_pred_xgb.csv', index = False)

## RandomForest: balanced

In [84]:
model_2 = random_forest_classifier_initialise()
model_2 = ensemble_model_initialise(base_estimator=model_2)
model_2.fit(tr_X, tr_y)

# -- Model saving
# pickle.dump(model_2, open('./models/classifier/RandomForest.pkl', 'wb'))
pickle.dump(model_2, open('./models/classifier/RandomForest_1.pkl', 'wb'))

model_2

EasyEnsembleClassifier(base_estimator=RandomForestClassifier(bootstrap=True,
                                                             class_weight='balanced',
                                                             criterion='gini',
                                                             max_depth=8,
                                                             max_features=20,
                                                             max_leaf_nodes=None,
                                                             min_impurity_decrease=0.0,
                                                             min_impurity_split=None,
                                                             min_samples_leaf=1,
                                                             min_samples_split=2,
                                                             min_weight_fraction_leaf=0.0,
                                                             n_estimators=500,
                 

In [85]:
# Validation
vL_y_pred = model_2.predict(vl_X)
vL_y_prob = model_2.predict_proba(vl_X)

cm = confusion_matrix(vl_y.values, vL_y_pred)
precision, recall, f1score, _ = precision_recall_fscore_support(vl_y.values, vL_y_pred, average='binary',
                                                                sample_weight=[class_weights[i] for i in vl_y.values])  # weighted
roc_area = roc_auc_score(vl_y.values, vL_y_prob[:, 1])
print('Confusion matrix: ', cm)
print('Precision: {p:.3f}'.format(p=precision))
print('Recall: {r:.3f}'.format(r=recall))
print('F1score: {f:.3f}'.format(f=f1score))
print('ROC curve under area: {roc:.3f}'.format(roc=roc_area))

Confusion matrix:  [[437401  13029]
 [   168   5939]]
Precision: 0.971
Recall: 0.972
F1score: 0.972
ROC curve under area: 0.995


In [21]:
# # Validation
# vL_y_pred = model_2.predict(vl_X)
# vL_y_prob = model_2.predict_proba(vl_X)

# cm = confusion_matrix(vl_y.values, vL_y_pred)
# precision, recall, f1score, _ = precision_recall_fscore_support(vl_y.values, vL_y_pred, average='binary',
#                                                                 sample_weight=[class_weights[i] for i in vl_y.values])  # weighted
# roc_area = roc_auc_score(vl_y.values, vL_y_prob[:, 1])
# print('Confusion matrix: ', cm)
# print('Precision: {p:.3f}'.format(p=precision))
# print('Recall: {r:.3f}'.format(r=recall))
# print('F1score: {f:.3f}'.format(f=f1score))
# print('ROC curve under area: {roc:.3f}'.format(roc=roc_area))

Confusion matrix:  [[145841   4338]
 [    51   1949]]
Precision: 0.971
Recall: 0.975
F1score: 0.973
ROC curve under area: 0.995


In [88]:
prob_rf =model_2.predict_proba(df_test_trans) 

prob_rf_list = []
for i in range(0, len(prob_rf)):
    prob_rf_list.append(prob_rf[i][1])

df_prob_rf = pd.DataFrame(prob_rf_list,columns = ['prob'])
df_prob_rf.values

array([[0.03462638],
       [0.03637189],
       [0.03440176],
       ...,
       [0.01024959],
       [0.01025589],
       [0.01262927]])

In [98]:
pred_rf = np.where((df_prob_rf.values > 0.875 ) == False, 0, 1 )
print(100*sum(pred_rf)/len(pred_rf))
print(sum(pred_rf))
print(len(pred_rf))
print(421665*(1.33/100)) # percentage of target=1 from training data: 5608.1445

[1.33044004]
[5610]
421665
5608.1445


In [99]:
fraud_ind_list= []
for i in range(0,len(pred_rf)):
    fraud_ind_list.append(pred_rf[i][0])
    
result_model_2 = pd.DataFrame(txkey,columns = ['txkey'])
result_model_2['fraud_ind'] = fraud_ind_list

result_model_2.to_csv(path + 'result/result_pred_rf_cutprob.csv', index = False)

## Logistic Regression: balanced

In [36]:
model_3 = logstic_initialise()
model_3 = ensemble_model_initialise(base_estimator=model_3)
model_3.fit(tr_X, tr_y)

# -- Model saving
pickle.dump(model_3, open('./models/classifier/LogsticReg.pkl', 'wb'))

model_3

EasyEnsembleClassifier(base_estimator=LogisticRegression(C=1.0,
                                                         class_weight='balanced',
                                                         dual=False,
                                                         fit_intercept=True,
                                                         intercept_scaling=1,
                                                         l1_ratio=None,
                                                         max_iter=2000,
                                                         multi_class='ovr',
                                                         n_jobs=-1,
                                                         penalty='l2',
                                                         random_state=None,
                                                         solver='saga',
                                                         tol=0.0001, verbose=0,
                                           

In [37]:
# Validation
vL_y_pred = model_3.predict(vl_X)
vL_y_prob = model_3.predict_proba(vl_X)

cm = confusion_matrix(vl_y.values, vL_y_pred)
precision, recall, f1score, _ = precision_recall_fscore_support(vl_y.values, vL_y_pred, average='binary',
                                                                sample_weight=[class_weights[i] for i in vl_y.values])  # weighted
roc_area = roc_auc_score(vl_y.values, vL_y_prob[:, 1])
print('Confusion matrix: ', cm)
print('Precision: {p:.3f}'.format(p=precision))
print('Recall: {r:.3f}'.format(r=recall))
print('F1score: {f:.3f}'.format(f=f1score))
print('ROC curve under area: {roc:.3f}'.format(roc=roc_area))

Confusion matrix:  [[144602   5577]
 [    76   1924]]
Precision: 0.962
Recall: 0.962
F1score: 0.962
ROC curve under area: 0.992


In [51]:
# load Model
# model_3 = pickle.load(open('./models/classifier/LogsticReg.pkl', "rb"))
prob_LR =model_3.predict_proba(df_test_trans) 

prob_LR_list = []
for i in range(0, len(prob_LR)):
    prob_LR_list.append(prob_LR[i][1])

df_prob_LR = pd.DataFrame(prob_LR_list,columns = ['prob'])
df_prob_LR.values

array([[0.01182728],
       [0.01286801],
       [0.01111791],
       ...,
       [0.00673416],
       [0.00853615],
       [0.01607185]])

In [77]:
pred_LR = np.where((df_prob_LR.values > 0.759 ) == False, 0, 1 )
print(100*sum(pred_LR)/len(pred_LR))
print(sum(pred_LR))
print(len(pred_LR))
print(421665*(1.33/100)) # percentage of target=1 from training data: 5608.1445

[1.33091435]
[5612]
421665
5608.1445


In [82]:
# fraud_ind_list= []
# for i in range(0,len(pred_LR)):
#     fraud_ind_list.append(pred_LR[i][0])
    
result_model_3 = pd.DataFrame(txkey,columns = ['txkey'])
result_model_3['fraud_ind'] = fraud_ind_list

# result_model_3.to_csv(path + 'result/result_pred_LR.csv', index = False)
result_model_3.to_csv(path + 'result/result_pred_LR_cutprob.csv', index = False)

## One Class SVM: balanced

In [41]:
model_4 = one_class_svm()
model_4 = ensemble_model_initialise(base_estimator=model_4)
model_4.fit(tr_X, tr_y)

# -- Model saving
pickle.dump(model_4, open('./models/classifier/one_class_svm.pkl', 'wb'))

model_4

EasyEnsembleClassifier(base_estimator=OneClassSVM(cache_size=200, coef0=0.0,
                                                  degree=3,
                                                  gamma='auto_deprecated',
                                                  kernel='rbf', max_iter=-1,
                                                  nu=0.5, random_state=None,
                                                  shrinking=True, tol=0.0001,
                                                  verbose=False),
                       n_estimators=50, n_jobs=-1, random_state=42,
                       replacement=False, sampling_strategy='auto', verbose=0,
                       warm_start=False)

In [42]:
# Validation
vL_y_pred = model_4.predict(vl_X)
vL_y_prob = model_4.predict_proba(vl_X)

cm = confusion_matrix(vl_y.values, vL_y_pred)
precision, recall, f1score, _ = precision_recall_fscore_support(vl_y.values, vL_y_pred, average='binary',
                                                                sample_weight=[class_weights[i] for i in vl_y.values])  # weighted
roc_area = roc_auc_score(vl_y.values, vL_y_prob[:, 1])
print('Confusion matrix: ', cm)
print('Precision: {p:.3f}'.format(p=precision))
print('Recall: {r:.3f}'.format(r=recall))
print('F1score: {f:.3f}'.format(f=f1score))
print('ROC curve under area: {roc:.3f}'.format(roc=roc_area))

Confusion matrix:  [[     0 150179]
 [     0   2000]]
Precision: 0.496
Recall: 1.000
F1score: 0.663
ROC curve under area: 0.500


In [43]:
pred_OC_SVM =model_4.predict(df_test_trans) 

result_model_4 = pd.DataFrame(txkey,columns = ['txkey'])
result_model_4['fraud_ind'] = pred_OC_SVM.tolist()

result_model_4.to_csv(path + 'result/result_pred_OC_SVM.csv', index = False)

## SVC: balanced

In [33]:
model_5 = svc_initialise()
model_5 = ensemble_model_initialise(base_estimator=model_5)
model_5.fit(tr_X, tr_y)

# -- Model saving
pickle.dump(model_5, open('./models/classifier/svc.pkl', 'wb'))

model_5

EasyEnsembleClassifier(base_estimator=SVC(C=25, cache_size=200,
                                          class_weight=None, coef0=0.0,
                                          decision_function_shape='ovr',
                                          degree=3, gamma=0.001, kernel='rbf',
                                          max_iter=-1, probability=False,
                                          random_state=None, shrinking=False,
                                          tol=0.001, verbose=False),
                       n_estimators=50, n_jobs=-1, random_state=42,
                       replacement=False, sampling_strategy='auto', verbose=0,
                       warm_start=False)

In [11]:
# -- Load Model
model_5 = pickle.load(open('./models/classifier/svc.pkl', 'rb'))

# Validation: confusion matrix of training data
vL_y_pred = model_5.predict(tr_X)
vL_y_prob = model_5.predict_proba(tr_X)

cm = confusion_matrix(tr_y.values, vL_y_pred)
precision, recall, f1score, _ = precision_recall_fscore_support(tr_y.values, vL_y_pred, average='binary',
                                                                sample_weight=[class_weights[i] for i in tr_y.values])  # weighted
roc_area = roc_auc_score(tr_y.values, vL_y_prob[:, 1])
print('Confusion matrix: ', cm)
print('Precision: {p:.3f}'.format(p=precision))
print('Recall: {r:.3f}'.format(r=recall))
print('F1score: {f:.3f}'.format(f=f1score))
print('ROC curve under area: {roc:.3f}'.format(roc=roc_area))

Confusion matrix:  [[1020951   30051]
 [    616   13632]]
Precision: 0.971
Recall: 0.957
F1score: 0.964
ROC curve under area: 0.969


In [34]:
# Validation: confusion matrix of validation data
vL_y_pred = model_5.predict(vl_X)
vL_y_prob = model_5.predict_proba(vl_X)

cm = confusion_matrix(vl_y.values, vL_y_pred)
precision, recall, f1score, _ = precision_recall_fscore_support(vl_y.values, vL_y_pred, average='binary',
                                                                sample_weight=[class_weights[i] for i in vl_y.values])  # weighted
roc_area = roc_auc_score(vl_y.values, vL_y_prob[:, 1])
print('Confusion matrix: ', cm)
print('Precision: {p:.3f}'.format(p=precision))
print('Recall: {r:.3f}'.format(r=recall))
print('F1score: {f:.3f}'.format(f=f1score))
print('ROC curve under area: {roc:.3f}'.format(roc=roc_area))

Confusion matrix:  [[437508  12922]
 [   285   5822]]
Precision: 0.971
Recall: 0.953
F1score: 0.962
ROC curve under area: 0.969


In [20]:
prob_SVC =model_5.predict_proba(df_test_trans) 

prob_SVC_list = []
for i in range(0, len(prob_SVC)):
    prob_SVC_list.append(prob_SVC[i][1])

df_prob_SVM = pd.DataFrame(prob_SVC_list,columns = ['prob'])
df_prob_SVM.values

array([[0.],
       [0.],
       [0.],
       ...,
       [0.],
       [0.],
       [0.]])

In [36]:
# pred_SVC= np.where((prob_SVC > 0.3) ==False, 0, 1)
pred_SVC= np.where((df_prob_SVM.values > 0.99999) == False, 0, 1)
print(100*sum(pred_SVC)/len(pred_SVC))
print(sum(pred_SVC))
print(len(pred_SVC))
print(421665*(1.33/100)) # percentage of target=1 from training data: 5608.1445

[1.86214175]
[7852]
421665
5608.1445


In [45]:
fraud_ind_list= []
for i in range(0,len(pred_SVC)):
    fraud_ind_list.append(pred_SVC[i][0])

result_model_5 = pd.DataFrame(txkey,columns = ['txkey'])
result_model_5['fraud_ind'] = fraud_ind_list

# # result_model_5.to_csv(path + 'result/result_pred_SVC.csv', index = False)
result_model_5.to_csv(path + 'result/result_pred_SVC_cutprob.csv', index = False)

In [38]:
# 跑22小時,仍沒跑完
# from sklearn.model_selection import GridSearchCV, RandomizedSearchCV 

# svc = SVC()

# params_svc = {'kernel':['rbf', 'sigmoid'],
#               'C':[0.001, 0.10, 0.1, 10, 25, 50, 100],
#               'gamma': [1e-2, 1e-3, 1e-4, 1e-5],
#               'shrinking':[True,False]}

# hyper_svc = RandomizedSearchCV(estimator = svc, param_distributions = params_svc, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
# hyper_svc.fit(X, y)

# print('\n Best estimator:')
# print(hyper_svc.best_estimator_)
# print('\n Best hyperparameters:')
# print(hyper_svc.best_params_)
# results = pd.DataFrame(hyper_svc.cv_results_)
# results.to_csv('./models/classifier/grid-search-svc-results-01.csv', index=False)

## NN(imbalanced)

In [7]:
from keras.models import Sequential
from keras.layers import Dense

Using TensorFlow backend.


In [ ]:
# classifier = Sequential()
# #First Hidden Layer
# classifier.add(Dense(4, activation='relu', kernel_initializer='random_normal', input_dim= len(tr_X.columns)))
# #Second  Hidden Layer
# classifier.add(Dense(4, activation='relu', kernel_initializer='random_normal'))
# #Output Layer
# classifier.add(Dense(1, activation='sigmoid', kernel_initializer='random_normal'))

# Confusion matrix:  [[448835   1595]
#  [  1657   4450]]
# Precision: 0.995
# Recall: 0.729
# F1score: 0.841
# ROC curve under area: 0.992

In [ ]:
# classifier = Sequential()
# #First Hidden Layer
# classifier.add(Dense(10, activation='relu', kernel_initializer='random_normal', input_dim= len(tr_X.columns)))
# #Second  Hidden Layer
# classifier.add(Dense(10, activation='relu', kernel_initializer='random_normal'))
# #Output Layer
# classifier.add(Dense(1, activation='sigmoid', kernel_initializer='random_normal'))

# #Compiling the neural network
# classifier.compile(optimizer ='adam',loss='binary_crossentropy', metrics =['accuracy'])

# #Fitting the data to the training dataset
# classifier.fit(tr_X,tr_y, batch_size=2300, epochs=700) 

# eval_model=classifier.evaluate(tr_X, tr_y)
# eval_model
# 1065250/1065250 [==============================] - 18s 17us/step
# [0.01645704336325073, 0.9940633773803711]



# cm = confusion_matrix(vl_y.values, y_pred_1)
# precision, recall, f1score, _ = precision_recall_fscore_support(vl_y.values, y_pred_1, average='binary',
#                                                                 sample_weight=[class_weights[i] for i in vl_y.values])  # weighted
# roc_area = roc_auc_score(vl_y.values, y_prob[:, 0])
# print('Confusion matrix: ', cm)
# print('Precision: {p:.3f}'.format(p=precision))
# print('Recall: {r:.3f}'.format(r=recall))
# print('F1score: {f:.3f}'.format(f=f1score))
# print('ROC curve under area: {roc:.3f}'.format(roc=roc_area))

# Confusion matrix:  [[448959   1471]
#  [  1490   4617]]
# Precision: 0.996
# Recall: 0.756
# F1score: 0.859
# ROC curve under area: 0.994

In [ ]:
classifier = Sequential()
#First Hidden Layer
# classifier.add(Dense(4, activation='relu', kernel_initializer='random_normal', input_dim= len(tr_X.columns)))
classifier.add(Dense(10, activation='relu', kernel_initializer='random_normal', input_dim= len(tr_X.columns)))
#Second  Hidden Layer
# classifier.add(Dense(4, activation='relu', kernel_initializer='random_normal'))
classifier.add(Dense(10, activation='relu', kernel_initializer='random_normal'))
#Output Layer
classifier.add(Dense(1, activation='sigmoid', kernel_initializer='random_normal'))

#Compiling the neural network
classifier.compile(optimizer ='adam',loss='binary_crossentropy', metrics =['accuracy'])

#Fitting the data to the training dataset
# classifier.fit(tr_X,tr_y, batch_size=20, epochs=100000) 
# classifier.fit(tr_X,tr_y, batch_size=2300, epochs=700) 
classifier.fit(tr_X,tr_y, batch_size=100, epochs=10000) 

Epoch 1/10000
1065250/1065250 [==============================] - 12s 11us/step - loss: 0.0294 - accuracy: 0.9899
Epoch 2/10000
1065250/1065250 [==============================] - 11s 11us/step - loss: 0.0203 - accuracy: 0.9924
Epoch 3/10000
1065250/1065250 [==============================] - 12s 11us/step - loss: 0.0193 - accuracy: 0.9928
Epoch 4/10000
1065250/1065250 [==============================] - 12s 11us/step - loss: 0.0187 - accuracy: 0.9930
Epoch 5/10000
1065250/1065250 [==============================] - 12s 11us/step - loss: 0.0184 - accuracy: 0.9931
Epoch 6/10000
1065250/1065250 [==============================] - 12s 11us/step - loss: 0.0182 - accuracy: 0.9933
Epoch 7/10000
1065250/1065250 [==============================] - 12s 11us/step - loss: 0.0181 - accuracy: 0.9934
Epoch 8/10000
1065250/1065250 [==============================] - 11s 11us/step - loss: 0.0180 - accuracy: 0.9934
Epoch 9/10000
1065250/1065250 [==============================] - 11s 11us/step - loss: 0.0179 - 

1065250/1065250 [==============================] - 11s 10us/step - loss: 0.0165 - accuracy: 0.9941
Epoch 144/10000
1065250/1065250 [==============================] - 11s 11us/step - loss: 0.0164 - accuracy: 0.9941
Epoch 145/10000
1065250/1065250 [==============================] - 12s 11us/step - loss: 0.0164 - accuracy: 0.9941
Epoch 146/10000
1065250/1065250 [==============================] - 12s 11us/step - loss: 0.0164 - accuracy: 0.9941
Epoch 147/10000
1065250/1065250 [==============================] - 11s 11us/step - loss: 0.0164 - accuracy: 0.9941
Epoch 148/10000
1065250/1065250 [==============================] - 11s 11us/step - loss: 0.0164 - accuracy: 0.9941
Epoch 149/10000
1065250/1065250 [==============================] - 11s 11us/step - loss: 0.0164 - accuracy: 0.9941
Epoch 150/10000
1065250/1065250 [==============================] - 11s 11us/step - loss: 0.0164 - accuracy: 0.9941
Epoch 151/10000
1065250/1065250 [==============================] - 11s 10us/step - loss: 0.0164 

1065250/1065250 [==============================] - 11s 11us/step - loss: 0.0163 - accuracy: 0.9941
Epoch 286/10000
1065250/1065250 [==============================] - 12s 11us/step - loss: 0.0162 - accuracy: 0.9941
Epoch 287/10000
1065250/1065250 [==============================] - 11s 11us/step - loss: 0.0162 - accuracy: 0.9942
Epoch 288/10000
1065250/1065250 [==============================] - 12s 11us/step - loss: 0.0162 - accuracy: 0.9942
Epoch 289/10000
1065250/1065250 [==============================] - 12s 11us/step - loss: 0.0162 - accuracy: 0.9942
Epoch 290/10000
1065250/1065250 [==============================] - 11s 11us/step - loss: 0.0162 - accuracy: 0.9942
Epoch 291/10000
1065250/1065250 [==============================] - 12s 11us/step - loss: 0.0162 - accuracy: 0.9942
Epoch 292/10000
1065250/1065250 [==============================] - 11s 11us/step - loss: 0.0162 - accuracy: 0.9942
Epoch 293/10000
1065250/1065250 [==============================] - 12s 11us/step - loss: 0.0162 

1065250/1065250 [==============================] - 11s 11us/step - loss: 0.0161 - accuracy: 0.9942
Epoch 428/10000
1065250/1065250 [==============================] - 12s 11us/step - loss: 0.0161 - accuracy: 0.9942
Epoch 429/10000
1065250/1065250 [==============================] - 12s 11us/step - loss: 0.0161 - accuracy: 0.9942
Epoch 430/10000
1065250/1065250 [==============================] - 11s 10us/step - loss: 0.0161 - accuracy: 0.9941
Epoch 431/10000
1065250/1065250 [==============================] - 12s 11us/step - loss: 0.0161 - accuracy: 0.9942
Epoch 432/10000
1065250/1065250 [==============================] - 12s 11us/step - loss: 0.0161 - accuracy: 0.9942
Epoch 433/10000
1065250/1065250 [==============================] - 11s 10us/step - loss: 0.0161 - accuracy: 0.9941
Epoch 434/10000
1065250/1065250 [==============================] - 12s 11us/step - loss: 0.0161 - accuracy: 0.9942
Epoch 435/10000
1065250/1065250 [==============================] - 11s 11us/step - loss: 0.0161 

1065250/1065250 [==============================] - 12s 11us/step - loss: 0.0161 - accuracy: 0.9942
Epoch 570/10000
1065250/1065250 [==============================] - 12s 11us/step - loss: 0.0161 - accuracy: 0.9942
Epoch 571/10000
1065250/1065250 [==============================] - 12s 11us/step - loss: 0.0161 - accuracy: 0.9942
Epoch 572/10000
1065250/1065250 [==============================] - 12s 11us/step - loss: 0.0161 - accuracy: 0.9942
Epoch 573/10000
1065250/1065250 [==============================] - 11s 11us/step - loss: 0.0161 - accuracy: 0.9942
Epoch 574/10000
1065250/1065250 [==============================] - 12s 11us/step - loss: 0.0161 - accuracy: 0.9942
Epoch 575/10000
1065250/1065250 [==============================] - 12s 11us/step - loss: 0.0161 - accuracy: 0.9942
Epoch 576/10000
1065250/1065250 [==============================] - 11s 10us/step - loss: 0.0161 - accuracy: 0.9942
Epoch 577/10000
1065250/1065250 [==============================] - 11s 11us/step - loss: 0.0161 

...

1065250/1065250 [==============================] - 11s 10us/step - loss: 0.0161 - accuracy: 0.9942
Epoch 4076/10000
1065250/1065250 [==============================] - 12s 11us/step - loss: 0.0161 - accuracy: 0.9942
Epoch 4077/10000
1065250/1065250 [==============================] - 12s 11us/step - loss: 0.0161 - accuracy: 0.9942
Epoch 4078/10000
1065250/1065250 [==============================] - 12s 11us/step - loss: 0.0161 - accuracy: 0.9942
Epoch 4079/10000
1065250/1065250 [==============================] - 12s 11us/step - loss: 0.0161 - accuracy: 0.9942
Epoch 4080/10000
1065250/1065250 [==============================] - 12s 11us/step - loss: 0.0161 - accuracy: 0.9942
Epoch 4081/10000
1065250/1065250 [==============================] - 12s 11us/step - loss: 0.0161 - accuracy: 0.9942
Epoch 4082/10000
1065250/1065250 [==============================] - 12s 11us/step - loss: 0.0161 - accuracy: 0.9942
Epoch 4083/10000
1065250/1065250 [==============================] - 11s 11us/step - loss:

1065250/1065250 [==============================] - 12s 11us/step - loss: 0.0161 - accuracy: 0.9942
Epoch 4216/10000
1065250/1065250 [==============================] - 12s 11us/step - loss: 0.0161 - accuracy: 0.9942
Epoch 4217/10000
1065250/1065250 [==============================] - 11s 11us/step - loss: 0.0161 - accuracy: 0.9942
Epoch 4218/10000
1065250/1065250 [==============================] - 12s 11us/step - loss: 0.0161 - accuracy: 0.9942
Epoch 4219/10000
1065250/1065250 [==============================] - 12s 11us/step - loss: 0.0161 - accuracy: 0.9942
Epoch 4220/10000
1065250/1065250 [==============================] - 12s 11us/step - loss: 0.0161 - accuracy: 0.9942
Epoch 4221/10000
1065250/1065250 [==============================] - 11s 11us/step - loss: 0.0161 - accuracy: 0.9942
Epoch 4222/10000
1065250/1065250 [==============================] - 12s 11us/step - loss: 0.0161 - accuracy: 0.9942
Epoch 4223/10000
1065250/1065250 [==============================] - 12s 11us/step - loss:

1065250/1065250 [==============================] - 12s 11us/step - loss: 0.0161 - accuracy: 0.9942
Epoch 4356/10000
1065250/1065250 [==============================] - 12s 11us/step - loss: 0.0161 - accuracy: 0.9942
Epoch 4357/10000
1065250/1065250 [==============================] - 12s 11us/step - loss: 0.0161 - accuracy: 0.9942
Epoch 4358/10000
1065250/1065250 [==============================] - 11s 11us/step - loss: 0.0161 - accuracy: 0.9942
Epoch 4359/10000
1065250/1065250 [==============================] - 11s 11us/step - loss: 0.0161 - accuracy: 0.9942
Epoch 4360/10000
1065250/1065250 [==============================] - 12s 11us/step - loss: 0.0161 - accuracy: 0.9943
Epoch 4361/10000
1065250/1065250 [==============================] - 12s 11us/step - loss: 0.0161 - accuracy: 0.9942
Epoch 4362/10000
1065250/1065250 [==============================] - 12s 11us/step - loss: 0.0161 - accuracy: 0.9942
Epoch 4363/10000
1065250/1065250 [==============================] - 12s 11us/step - loss:

1065250/1065250 [==============================] - 12s 11us/step - loss: 0.0161 - accuracy: 0.9942
Epoch 4496/10000
1065250/1065250 [==============================] - 11s 11us/step - loss: 0.0161 - accuracy: 0.9942
Epoch 4497/10000
1065250/1065250 [==============================] - 12s 11us/step - loss: 0.0161 - accuracy: 0.9942
Epoch 4498/10000
1065250/1065250 [==============================] - 12s 11us/step - loss: 0.0161 - accuracy: 0.9942
Epoch 4499/10000
1065250/1065250 [==============================] - 12s 11us/step - loss: 0.0160 - accuracy: 0.9942
Epoch 4500/10000
1065250/1065250 [==============================] - 12s 11us/step - loss: 0.0160 - accuracy: 0.9942
Epoch 4501/10000
1065250/1065250 [==============================] - 12s 11us/step - loss: 0.0161 - accuracy: 0.9942
Epoch 4502/10000
1065250/1065250 [==============================] - 12s 11us/step - loss: 0.0161 - accuracy: 0.9942
Epoch 4503/10000
1065250/1065250 [==============================] - 12s 11us/step - loss:

1065250/1065250 [==============================] - 12s 11us/step - loss: 0.0161 - accuracy: 0.9942
Epoch 4636/10000
1065250/1065250 [==============================] - 12s 11us/step - loss: 0.0161 - accuracy: 0.9942
Epoch 4637/10000
1065250/1065250 [==============================] - 12s 11us/step - loss: 0.0161 - accuracy: 0.9942
Epoch 4638/10000
1065250/1065250 [==============================] - 11s 10us/step - loss: 0.0160 - accuracy: 0.9942
Epoch 4639/10000
1065250/1065250 [==============================] - 12s 11us/step - loss: 0.0161 - accuracy: 0.9942
Epoch 4640/10000
1065250/1065250 [==============================] - 12s 11us/step - loss: 0.0161 - accuracy: 0.9942
Epoch 4641/10000
1065250/1065250 [==============================] - 11s 11us/step - loss: 0.0160 - accuracy: 0.9942
Epoch 4642/10000
1065250/1065250 [==============================] - 12s 11us/step - loss: 0.0161 - accuracy: 0.9942
Epoch 4643/10000
1065250/1065250 [==============================] - 12s 11us/step - loss:

In [ ]:
# -- Model saving
# pickle.dump(classifier, open('./models/classifier/NN_nobalance_epochs729.pkl', 'wb'))
# pickle.dump(classifier, open('./models/classifier/NN_nobalance_epochs700.pkl', 'wb'))
pickle.dump(classifier, open('./models/classifier/NN_nobalance_epochs10000.pkl', 'wb'))
classifier

In [ ]:
eval_model=classifier.evaluate(tr_X, tr_y)
eval_model

In [145]:
# -- load
# classifier = pickle.load(open('./models/classifier/NN_nobalance_epochs729.pkl', "rb"))
y_prob =classifier.predict(vl_X)
y_pred_1 = np.where((y_prob > 0.38) == False, 0, 1 )

print(len(vl_X)*((1.33/100)))
print(sum(y_pred_1))

6071.9421
[6088]


In [146]:
cm = confusion_matrix(vl_y.values, y_pred_1)
precision, recall, f1score, _ = precision_recall_fscore_support(vl_y.values, y_pred_1, average='binary',
                                                                sample_weight=[class_weights[i] for i in vl_y.values])  # weighted
roc_area = roc_auc_score(vl_y.values, y_prob[:, 0])
print('Confusion matrix: ', cm)
print('Precision: {p:.3f}'.format(p=precision))
print('Recall: {r:.3f}'.format(r=recall))
print('F1score: {f:.3f}'.format(f=f1score))
print('ROC curve under area: {roc:.3f}'.format(roc=roc_area))

Confusion matrix:  [[448959   1471]
 [  1490   4617]]
Precision: 0.996
Recall: 0.756
F1score: 0.859
ROC curve under area: 0.994


In [162]:
prob_NN =classifier.predict(df_test_trans) 
pred_NN = np.where((prob_NN > 0.0458) == False, 0, 1 )

print(100*sum(pred_NN)/len(pred_NN))
print(sum(pred_NN))
print(len(pred_NN))
print(421665*(1.33/100)) # percentage of target=1 from training data

[1.3285428]
[5602]
421665
5608.1445


In [163]:
pred_NN_list = []
for i in range(0, len(pred_NN)):
    pred_NN_list.append(pred_NN[i][0])

result_model_6 = pd.DataFrame(txkey,columns = ['txkey'])
result_model_6['fraud_ind'] = pred_NN_list

# result_model_6.to_csv(path + 'result/result_pred_NN_nobalance_epochs729.csv', index = False)
result_model_6.to_csv(path + 'result/result_pred_NN_nobalance_epochs700.csv', index = False)

In [ ]:
SUM()

## NN(balance)

In [8]:
from imblearn.over_sampling import SMOTE, ADASYN
from collections import Counter

In [107]:
X_resampled, y_resampled = SMOTE().fit_resample(X, y)
print(sorted(Counter(y_resampled).items()))

[(0, 1501432), (1, 1501432)]


In [113]:
# Training/validation set split
# tr_X_rs, vl_X_rs, tr_y_rs, vl_y_rs = train_test_split(X_resampled, y_resampled, test_size=0.3, stratify = y_resampled, random_state=42)

print(len(tr_X_rs))
print(len(tr_y_rs))
print(len(vl_X_rs))
print(len(vl_y_rs))

2102004
2102004
900860
900860


In [ ]:
# clf_smote = LinearSVC().fit(X_resampled, y_resampled)

In [66]:
class_weight = {1: 0.98,
                0: 0.02}

In [114]:
classifier = Sequential()
#First Hidden Layer
classifier.add(Dense(10, activation='relu', kernel_initializer='random_normal', input_dim= len(tr_X.columns)))
#Second  Hidden Layer
classifier.add(Dense(10, activation='relu', kernel_initializer='random_normal'))
#Output Layer
classifier.add(Dense(1, activation='sigmoid', kernel_initializer='random_normal'))

#Compiling the neural network
classifier.compile(optimizer ='adam',loss='binary_crossentropy', metrics =['accuracy'])

#Fitting the data to the training dataset
# classifier.fit(tr_X,tr_y, batch_size=2300, epochs=700, class_weight = class_weight) 
classifier.fit(tr_X_rs,tr_y_rs, batch_size=2300, epochs=700) 

Epoch 1/700
2102004/2102004 [==============================] - 3s 1us/step - loss: 0.1737 - accuracy: 0.9447
Epoch 2/700
2102004/2102004 [==============================] - 3s 1us/step - loss: 0.1012 - accuracy: 0.9645
Epoch 3/700
2102004/2102004 [==============================] - 3s 1us/step - loss: 0.0950 - accuracy: 0.9666
Epoch 4/700
2102004/2102004 [==============================] - 3s 1us/step - loss: 0.0921 - accuracy: 0.9676
Epoch 5/700
2102004/2102004 [==============================] - 3s 1us/step - loss: 0.0902 - accuracy: 0.9683
Epoch 6/700
2102004/2102004 [==============================] - 3s 1us/step - loss: 0.0879 - accuracy: 0.9691
Epoch 7/700
2102004/2102004 [==============================] - 3s 1us/step - loss: 0.0861 - accuracy: 0.9698
Epoch 8/700
2102004/2102004 [==============================] - 3s 1us/step - loss: 0.0847 - accuracy: 0.9702
Epoch 9/700
2102004/2102004 [==============================] - 3s 1us/step - loss: 0.0837 - accuracy: 0.9705
Epoch 10/700
210200

2102004/2102004 [==============================] - 3s 1us/step - loss: 0.0710 - accuracy: 0.9749
Epoch 150/700
2102004/2102004 [==============================] - 3s 1us/step - loss: 0.0711 - accuracy: 0.9749
Epoch 151/700
2102004/2102004 [==============================] - 3s 1us/step - loss: 0.0710 - accuracy: 0.9750
Epoch 152/700
2102004/2102004 [==============================] - 3s 1us/step - loss: 0.0711 - accuracy: 0.9749
Epoch 153/700
2102004/2102004 [==============================] - 3s 1us/step - loss: 0.0710 - accuracy: 0.9750
Epoch 154/700
2102004/2102004 [==============================] - 3s 1us/step - loss: 0.0709 - accuracy: 0.9750
Epoch 155/700
2102004/2102004 [==============================] - 3s 1us/step - loss: 0.0710 - accuracy: 0.9750
Epoch 156/700
2102004/2102004 [==============================] - 3s 1us/step - loss: 0.0709 - accuracy: 0.9750
Epoch 157/700
2102004/2102004 [==============================] - 3s 1us/step - loss: 0.0710 - accuracy: 0.9750
Epoch 158/700
2

2102004/2102004 [==============================] - 2s 1us/step - loss: 0.0703 - accuracy: 0.9754
Epoch 296/700
2102004/2102004 [==============================] - 2s 1us/step - loss: 0.0703 - accuracy: 0.9754
Epoch 297/700
2102004/2102004 [==============================] - 2s 1us/step - loss: 0.0704 - accuracy: 0.9754
Epoch 298/700
2102004/2102004 [==============================] - 2s 1us/step - loss: 0.0703 - accuracy: 0.9754
Epoch 299/700
2102004/2102004 [==============================] - 2s 1us/step - loss: 0.0702 - accuracy: 0.9755
Epoch 300/700
2102004/2102004 [==============================] - 2s 1us/step - loss: 0.0702 - accuracy: 0.9754
Epoch 301/700
2102004/2102004 [==============================] - 2s 1us/step - loss: 0.0702 - accuracy: 0.9755
Epoch 302/700
2102004/2102004 [==============================] - 2s 1us/step - loss: 0.0701 - accuracy: 0.9755
Epoch 303/700
2102004/2102004 [==============================] - 2s 1us/step - loss: 0.0702 - accuracy: 0.9755
Epoch 304/700
2

2102004/2102004 [==============================] - 2s 1us/step - loss: 0.0695 - accuracy: 0.9758
Epoch 442/700
2102004/2102004 [==============================] - 2s 1us/step - loss: 0.0695 - accuracy: 0.9758
Epoch 443/700
2102004/2102004 [==============================] - 2s 1us/step - loss: 0.0695 - accuracy: 0.9758
Epoch 444/700
2102004/2102004 [==============================] - 2s 1us/step - loss: 0.0695 - accuracy: 0.9758
Epoch 445/700
2102004/2102004 [==============================] - 2s 1us/step - loss: 0.0694 - accuracy: 0.9758
Epoch 446/700
2102004/2102004 [==============================] - 2s 1us/step - loss: 0.0694 - accuracy: 0.9759
Epoch 447/700
2102004/2102004 [==============================] - 2s 1us/step - loss: 0.0695 - accuracy: 0.9757
Epoch 448/700
2102004/2102004 [==============================] - 2s 1us/step - loss: 0.0695 - accuracy: 0.9758
Epoch 449/700
2102004/2102004 [==============================] - 2s 1us/step - loss: 0.0694 - accuracy: 0.9758
Epoch 450/700
2

2102004/2102004 [==============================] - 2s 1us/step - loss: 0.0690 - accuracy: 0.9758
Epoch 588/700
2102004/2102004 [==============================] - 2s 1us/step - loss: 0.0691 - accuracy: 0.9758
Epoch 589/700
2102004/2102004 [==============================] - 2s 1us/step - loss: 0.0690 - accuracy: 0.9757
Epoch 590/700
2102004/2102004 [==============================] - 2s 1us/step - loss: 0.0690 - accuracy: 0.9758
Epoch 591/700
2102004/2102004 [==============================] - 2s 1us/step - loss: 0.0691 - accuracy: 0.9757
Epoch 592/700
2102004/2102004 [==============================] - 2s 1us/step - loss: 0.0691 - accuracy: 0.9758
Epoch 593/700
2102004/2102004 [==============================] - 2s 1us/step - loss: 0.0690 - accuracy: 0.9758
Epoch 594/700
2102004/2102004 [==============================] - 2s 1us/step - loss: 0.0690 - accuracy: 0.9759
Epoch 595/700
2102004/2102004 [==============================] - 2s 1us/step - loss: 0.0691 - accuracy: 0.9757
Epoch 596/700
2

In [115]:
# -- Model saving
# pickle.dump(classifier, open('./models/classifier/NN_balance_epochs700_ClassWeight.pkl', 'wb'))
pickle.dump(classifier, open('./models/classifier/NN_balance_epochs700_SMOTE.pkl', 'wb'))
classifier

In [69]:
# # "NN_balance_epochs700_ClassWeight.pkl"
# eval_model=classifier.evaluate(tr_X, tr_y)
# eval_model

1065250/1065250 [==============================] - 12s 11us/step


[0.06790028230451454, 0.9762553572654724]

In [79]:
# # "NN_balance_epochs700_ClassWeight.pkl"
# y_prob =classifier.predict(vl_X)
# y_pred_1 = np.where((y_prob > 0.38) == False, 0, 1 )

# print(len(vl_X)*((1.33/100)))
# print(sum(y_pred_1))

# cm = confusion_matrix(vl_y.values, y_pred_1)
# precision, recall, f1score, _ = precision_recall_fscore_support(vl_y.values, y_pred_1, average='binary',
#                                                                 sample_weight=[class_weights[i] for i in vl_y.values])  # weighted
# roc_area = roc_auc_score(vl_y.values, y_prob[:, 0])
# print('Confusion matrix: ', cm)
# print('Precision: {p:.3f}'.format(p=precision))
# print('Recall: {r:.3f}'.format(r=recall))
# print('F1score: {f:.3f}'.format(f=f1score))
# print('ROC curve under area: {roc:.3f}'.format(roc=roc_area))

6071.9421
[19590]
Confusion matrix:  [[436752  13678]
 [   195   5912]]
Precision: 0.970
Recall: 0.968
F1score: 0.969
ROC curve under area: 0.995


In [94]:
# # "NN_balance_epochs700_ClassWeight.pkl"
# prob_NN =classifier.predict(df_test_trans) 
# pred_NN = np.where((prob_NN > 0.7885) == False, 0, 1 )

# print(100*sum(pred_NN)/len(pred_NN))
# print(sum(pred_NN))
# print(len(pred_NN))
# print(421665*(1.33/100)) # percentage of target=1 from training data

[1.33304875]
[5621]
421665
5608.1445


In [116]:
# # "NN_balance_epochs700_SMOTE.pkl"
eval_model=classifier.evaluate(tr_X_rs, tr_y_rs)
eval_model

2102004/2102004 [==============================] - 25s 12us/step


[0.06886295653568866, 0.9756280183792114]

In [119]:
vl_y_rs

array([1, 0, 0, ..., 1, 0, 0], dtype=int64)

In [121]:
y_prob

array([[9.6676409e-01],
       [3.6001205e-05],
       [2.0861626e-07],
       ...,
       [9.9094301e-01],
       [3.4230146e-05],
       [4.1912459e-05]], dtype=float32)

In [123]:
df_y_prob

,prob
0,9.667641e-01
1,3.600121e-05
2,2.086163e-07
3,4.321337e-06
4,9.975061e-01
5,9.894614e-01
6,0.000000e+00
7,1.183152e-05
8,5.334616e-06
9,0.000000e+00


In [147]:
# # "NN_balance_epochs700_SMOTE.pkl"
y_prob =classifier.predict_proba(vl_X_rs)
prob_NN_list = []
for i in range(0, len(y_prob)):
    prob_NN_list.append(y_prob[i][0])

df_y_prob = pd.DataFrame(prob_NN_list,columns = ['prob'])
df_y_prob.values

array([[9.66764092e-01],
       [3.60012054e-05],
       [2.08616257e-07],
       ...,
       [9.90943015e-01],
       [3.42301464e-05],
       [4.19124590e-05]])

In [157]:
y_pred_1 = np.where((df_y_prob >= 0.5) == False, 0, 1 )
print(len(vl_X_rs)*((1.33/100)))
print(sum(y_pred_1))
print(len(vl_X_rs))
print(100*sum(y_pred_1)/len(vl_X_rs))      

11981.438
[451623]
900860
[50.13242901]


In [158]:
cm = confusion_matrix(vl_y_rs, y_pred_1)
precision, recall, f1score, _ = precision_recall_fscore_support(vl_y_rs, y_pred_1, average='binary',
                                                                sample_weight=[class_weights[i] for i in vl_y_rs])  # weighted
roc_area = roc_auc_score(vl_y_rs, y_prob[:, 0])
print('Confusion matrix: ', cm)
print('Precision: {p:.3f}'.format(p=precision))
print('Recall: {r:.3f}'.format(r=recall))
print('F1score: {f:.3f}'.format(f=f1score))
print('ROC curve under area: {roc:.3f}'.format(roc=roc_area))

Confusion matrix:  [[438873  11557]
 [ 10364 440066]]
Precision: 1.000
Recall: 0.977
F1score: 0.988
ROC curve under area: 0.996


In [176]:
# # "NN_balance_epochs700_SMOTE.pkl"
prob_NN =classifier.predict(df_test_trans) 
# pred_NN = np.where((prob_NN >= 0.5) == False, 0, 1 )
pred_NN = np.where((prob_NN > 0.9837) == False, 0, 1 )

print(100*sum(pred_NN)/len(pred_NN))
print(sum(pred_NN))
print(len(pred_NN))
print(421665*(1.33/100)) # percentage of target=1 from training data

[1.33779185]
[5641]
421665
5608.1445


In [177]:
pred_NN_list = []
for i in range(0, len(pred_NN)):
    pred_NN_list.append(pred_NN[i][0])

result_model_6 = pd.DataFrame(txkey,columns = ['txkey'])
result_model_6['fraud_ind'] = pred_NN_list

# result_model_6.to_csv(path + 'result/result_pred_NN_balance_epochs700.csv', index = False)
# result_model_6.to_csv(path + 'result/result_pred_NN_balance_epochs700_SMOTE.csv', index = False)
result_model_6.to_csv(path + 'result/result_pred_NN_balance_epochs700_SMOTE_1.csv', index = False)

## LGB(balanced)

In [9]:
model_7 = lgb_initialise()
model_7 = ensemble_model_initialise(base_estimator=model_7)
model_7.fit(tr_X, tr_y)

# -- Model saving
pickle.dump(model_7, open('./models/classifier/LGB.pkl', 'wb'))

model_7

EasyEnsembleClassifier(base_estimator=LGBMClassifier(boosting_type='gbdt',
                                                     class_weight=None,
                                                     colsample_bytree=0.7,
                                                     importance_type='split',
                                                     is_unbalance=True,
                                                     learning_rate=0.05,
                                                     max_depth=4,
                                                     min_child_samples=20,
                                                     min_child_weight=0.001,
                                                     min_split_gain=0.0,
                                                     n_estimators=600,
                                                     n_jobs=-1, nthread=3,
                                                     num_leaves=8,
                                                   

In [10]:
# Validation
vL_y_pred = model_7.predict(vl_X)
vL_y_prob = model_7.predict_proba(vl_X)

cm = confusion_matrix(vl_y.values, vL_y_pred)
precision, recall, f1score, _ = precision_recall_fscore_support(vl_y.values, vL_y_pred, average='binary',
                                                                sample_weight=[class_weights[i] for i in vl_y.values])  # weighted
roc_area = roc_auc_score(vl_y.values, vL_y_prob[:, 1])
print('Confusion matrix: ', cm)
print('Precision: {p:.3f}'.format(p=precision))
print('Recall: {r:.3f}'.format(r=recall))
print('F1score: {f:.3f}'.format(f=f1score))
print('ROC curve under area: {roc:.3f}'.format(roc=roc_area))

Confusion matrix:  [[436880  13550]
 [   158   5949]]
Precision: 0.970
Recall: 0.974
F1score: 0.972
ROC curve under area: 0.996


In [30]:
# -- load
model_7 = pickle.load(open('./models/classifier/LGB.pkl', "rb"))
prob_LGB = model_7.predict_proba(df_test_trans) 

prob_LGB_list = []
for i in range(0, len(prob_LGB)):
    prob_LGB_list.append(prob_LGB[i][1])

df_prob_LGB = pd.DataFrame(prob_LGB_list,columns = ['prob'])
df_prob_LGB.values

array([[0.08241862],
       [0.08757596],
       [0.08455157],
       ...,
       [0.00320813],
       [0.00395819],
       [0.00366722]])

In [54]:
pred_LGB = np.where((df_prob_LGB.values > 0.9145 ) == False, 0, 1 )
print(100*sum(pred_LGB)/len(pred_LGB))
print(sum(pred_LGB))
print(len(pred_LGB))
print(421665*(1.33/100)) # percentage of target=1 from training data: 5608.1445

[1.33494599]
[5629]
421665
5608.1445


In [60]:
fraud_ind_list= []
for i in range(0,len(pred_LGB)):
    fraud_ind_list.append(pred_LGB[i][0])

result_model_7 = pd.DataFrame(txkey,columns = ['txkey'])
result_model_7['fraud_ind'] = fraud_ind_list

result_model_7.to_csv(path + 'result/result_pred_LGB.csv', index = False)

## Voting Classifiers

In [9]:
from sklearn.ensemble import VotingClassifier

In [10]:
SVC_clf = pickle.load(open('./models/classifier/svc.pkl', "rb"))
NN_clf = pickle.load(open('./models/classifier/NN_nobalance_epochs700.pkl', "rb"))
LR_clf = pickle.load(open('./models/classifier/LogsticReg.pkl', "rb"))
LGB_clf = pickle.load(open('./models/classifier/LGB.pkl', "rb"))
XGB_clf = pickle.load(open('./models/classifier/ensemble_xgb_1.pkl', "rb"))
RF_clf = pickle.load(open('./models/classifier/RandomForest_1.pkl', "rb"))

### voting = 'hard'

In [16]:
voting_clf = VotingClassifier(
        estimators = [('svc', SVC_clf),('lr', LR_clf), ('rf', RF_clf)],
        voting = 'hard')
# ,('nn', NN_clf),('lgb', LGB_clf),('xgb', XGB_clf)
voting_clf.fit(tr_X, tr_y)

VotingClassifier(estimators=[('svc',
                              EasyEnsembleClassifier(base_estimator=SVC(C=25,
                                                                        cache_size=200,
                                                                        class_weight=None,
                                                                        coef0=0.0,
                                                                        decision_function_shape='ovr',
                                                                        degree=3,
                                                                        gamma=0.001,
                                                                        kernel='rbf',
                                                                        max_iter=-1,
                                                                        probability=False,
                                                                        random_state=None,


In [18]:
for clf in (LR_clf, RF_clf, SVC_clf, voting_clf):
    clf.fit(tr_X, tr_y)
    y_pred = clf.predict(vl_X)
    print(clf.__class__.__name__, accuracy_score(vl_y, y_pred))

EasyEnsembleClassifier 0.9637313076486681
EasyEnsembleClassifier 0.9710932520255752
EasyEnsembleClassifier 0.9710713479958908
VotingClassifier 0.9706639330437621


In [19]:
voting_clf_1 = VotingClassifier(
        estimators = [('svc', SVC_clf),('lr', LR_clf), ('rf', RF_clf),('lgb', LGB_clf),('xgb', XGB_clf)],
        voting = 'hard')
# ,('nn', NN_clf)
voting_clf_1.fit(tr_X, tr_y)

VotingClassifier(estimators=[('svc',
                              EasyEnsembleClassifier(base_estimator=SVC(C=25,
                                                                        cache_size=200,
                                                                        class_weight=None,
                                                                        coef0=0.0,
                                                                        decision_function_shape='ovr',
                                                                        degree=3,
                                                                        gamma=0.001,
                                                                        kernel='rbf',
                                                                        max_iter=-1,
                                                                        probability=False,
                                                                        random_state=None,


In [20]:
for clf in (LR_clf, RF_clf, SVC_clf, LGB_clf, XGB_clf, voting_clf_1):
    clf.fit(tr_X, tr_y)
    y_pred = clf.predict(vl_X)
    print(clf.__class__.__name__, accuracy_score(vl_y, y_pred))

EasyEnsembleClassifier 0.9637313076486681
EasyEnsembleClassifier 0.9710932520255752
EasyEnsembleClassifier 0.9710713479958908
EasyEnsembleClassifier 0.9699739561087053
EasyEnsembleClassifier 0.970981541474185
VotingClassifier 0.9715006669777039


In [22]:
# -- Model saving
pickle.dump(voting_clf, open('./models/classifier/voting_clf.pkl', 'wb'))
# -- Model saving
pickle.dump(voting_clf_1, open('./models/classifier/voting_clf_1.pkl', 'wb'))

In [25]:
# prob_voting = voting_clf_1.predict_proba(df_test_trans)
pred_voting = voting_clf_1.predict(df_test_trans)

In [26]:
print(100*sum(pred_voting)/len(pred_voting))
print(sum(pred_voting))
print(len(pred_voting))
print(421665*(1.33/100)) # percentage of target=1 from training data: 5608.1445

7.607935209230076
32080
421665
5608.1445


In [28]:
fraud_ind_list= []
for i in range(0,len(pred_voting)):
    fraud_ind_list.append(pred_voting[i][0])
    
result_voting_clf_1 = pd.DataFrame(txkey,columns = ['txkey'])
result_voting_clf_1['fraud_ind'] = pred_voting

result_voting_clf_1.to_csv(path + 'result/result_pred_voting_1.csv', index = False)

### voting = 'soft'

In [30]:
voting_clf_2 = VotingClassifier(
        estimators = [('svc', SVC_clf),('lr', LR_clf), ('rf', RF_clf),('lgb', LGB_clf),('xgb', XGB_clf)],
        voting = 'soft')
# ,('nn', NN_clf)
voting_clf_2.fit(tr_X, tr_y)

VotingClassifier(estimators=[('svc',
                              EasyEnsembleClassifier(base_estimator=SVC(C=25,
                                                                        cache_size=200,
                                                                        class_weight=None,
                                                                        coef0=0.0,
                                                                        decision_function_shape='ovr',
                                                                        degree=3,
                                                                        gamma=0.001,
                                                                        kernel='rbf',
                                                                        max_iter=-1,
                                                                        probability=False,
                                                                        random_state=None,


In [31]:
for clf in (LR_clf, RF_clf, SVC_clf, LGB_clf, XGB_clf, NN_clf, voting_clf_2):
    clf.fit(tr_X, tr_y)
    y_pred = clf.predict(vl_X)
    print(clf.__class__.__name__, accuracy_score(vl_y, y_pred))



EasyEnsembleClassifier 0.9637313076486681
EasyEnsembleClassifier 0.9710932520255752
EasyEnsembleClassifier 0.9710713479958908
EasyEnsembleClassifier 0.9699739561087053
EasyEnsembleClassifier 0.970981541474185
Epoch 1/1
1065250/1065250 [==============================] - 33s 31us/step - loss: 0.0175 - accuracy: 0.9937


ValueError: Classification metrics can't handle a mix of binary and continuous targets

In [32]:
# -- Model saving
pickle.dump(voting_clf_2, open('./models/classifier/voting_clf_2.pkl', 'wb'))

In [91]:
prob_voting2 = voting_clf_2.predict_proba(vl_X)
prob_voting2_list = []
for i in range(0, len(prob_voting2)):
    prob_voting2_list.append(prob_voting2[i][1])

df_prob_voting2 = pd.DataFrame(prob_voting2_list,columns = ['prob'])
df_prob_voting2.values

array([[0.00513025],
       [0.00257388],
       [0.00151946],
       ...,
       [0.00148027],
       [0.00160327],
       [0.00493177]])

In [112]:
pred_voting2 = np.where((df_prob_voting2.values >= 0.5 ) == False, 0, 1 )
print(100*sum(pred_voting2)/len(pred_voting2))
print(sum(pred_voting2))
print(len(pred_voting2))
print(421665*(1.33/100)) # percentage of target=1 from training data: 5608.1445

[4.01851329]
[18346]
456537
5608.1445


In [113]:
vL_y_pred_list= []
for i in range(0,len(pred_voting2)):
    vL_y_pred_list.append(pred_voting2[i][0])

cm = confusion_matrix(vl_y.values, vL_y_pred_list)
precision, recall, f1score, _ = precision_recall_fscore_support(vl_y.values, vL_y_pred_list, average='binary',
                                                                sample_weight=[class_weights[i] for i in vl_y.values])  # weighted
roc_area = roc_auc_score(vl_y.values, vL_y_pred_list)
print('Confusion matrix: ', cm)
print('Precision: {p:.3f}'.format(p=precision))
print('Recall: {r:.3f}'.format(r=recall))
print('F1score: {f:.3f}'.format(f=f1score))
print('ROC curve under area: {roc:.3f}'.format(roc=roc_area))

Confusion matrix:  [[437984  12446]
 [   207   5900]]
Precision: 0.972
Recall: 0.966
F1score: 0.969
ROC curve under area: 0.969


In [35]:
prob_voting_2 = voting_clf_2.predict_proba(df_test_trans)
prob_voting_2_list = []
for i in range(0, len(prob_voting_2)):
    prob_voting_2_list.append(prob_voting_2[i][1])

df_prob_voting_2 = pd.DataFrame(prob_voting_2_list,columns = ['prob'])
df_prob_voting_2.values

array([[0.04157406],
       [0.0564793 ],
       [0.04520682],
       ...,
       [0.0040177 ],
       [0.00470973],
       [0.00660974]])

In [114]:
# pred_voting_2 = np.where((df_prob_voting_2.values > 0.86 ) == False, 0, 1 )
pred_voting_2 = np.where((df_prob_voting_2.values >= 0.5 ) == False, 0, 1 )
print(100*sum(pred_voting_2)/len(pred_voting_2))
print(sum(pred_voting_2))
print(len(pred_voting_2))
print(421665*(1.33/100)) # percentage of target=1 from training data: 5608.1445

[5.30658224]
[22376]
421665
5608.1445


In [115]:
fraud_ind_list= []
for i in range(0,len(pred_voting_2)):
    fraud_ind_list.append(pred_voting_2[i][0])

result_voting_2 = pd.DataFrame(txkey,columns = ['txkey'])
result_voting_2['fraud_ind'] = fraud_ind_list

# result_voting_2.to_csv(path + 'result/result_pred_voting_2.csv', index = False) #np.where((df_prob_voting_2.values > 0.86 ) == False, 0, 1 )
result_voting_2.to_csv(path + 'result/result_pred_voting_2_1.csv', index = False) #np.where((df_prob_voting_2.values >= 0.5 ) == False, 0, 1 )